In [ ]:
import base64
from IPython.display import HTML, display, clear_output
from google.colab import output

# Define the path to the image
image_path = '/content/graph.png'

# Load the image data
with open(image_path, 'rb') as file:
    image_data = file.read()
    image_base64 = base64.b64encode(image_data).decode('utf-8')

# List to store clicked points and custom coordinates
clicked_points = []
custom_coordinates = []

# Define the limits of the custom coordinate system
x_min, x_max = 0, 10  # Example limits for width
y_min, y_max = 0, 0.1  # Example limits for height

# Define the HTML with the image data URI
def generate_html():
    dots_html = ''.join(
        f'<div class="dot" style="position:absolute;width:5px;height:5px;background-color:red;border-radius:50%;'
        f'left:{x}px;top:{y}px;"></div>' for x, y in clicked_points
    )

    return f'''
        <div id="image-container" style="position:relative; width:800px; height:auto; display: inline-block; vertical-align: top;">
            <img src="data:image/png;base64,{image_base64}" id="image" style="width:100%; height:auto; display:block; cursor:crosshair;">
            <div id="dots-container" style="position:absolute; top:0; left:0; width:100%; height:100%; pointer-events:none;">
                {dots_html}
            </div>
        </div>
        <button id="undo" style="margin-left: 10px;">Undo</button>
        <script>
            document.getElementById('image').addEventListener('click', function(event) {{
                const rect = event.target.getBoundingClientRect();
                const x = event.clientX - rect.left;
                const y = event.clientY - rect.top;

                // Notify Python of the click
                google.colab.kernel.invokeFunction('notebook.on_click', [x, y, rect.width, rect.height], {{}});

                // Add the dot dynamically without reloading the entire image
                const dot = document.createElement('div');
                dot.classList.add("dot");
                dot.style.position = 'absolute';
                dot.style.width = '3px';
                dot.style.height = '3px';
                dot.style.backgroundColor = 'red';
                dot.style.borderRadius = '50%';
                dot.style.left = x + 'px';
                dot.style.top = y + 'px';
                document.getElementById('dots-container').appendChild(dot);
            }});

            document.getElementById('undo').addEventListener('click', function() {{
                // Notify Python of the undo action
                google.colab.kernel.invokeFunction('notebook.on_undo', [], {{}});

                // Remove the last dot from the container dynamically
                const dotsContainer = document.getElementById('dots-container');
                if (dotsContainer.lastChild) {{
                    dotsContainer.removeChild(dotsContainer.lastChild);
                }}
            }});
        </script>
    '''

# Function to handle click events
def on_click(x, y, img_width, img_height):
    # Calculate the scale factors
    x_scale = (x_max - x_min) / img_width
    y_scale = (y_max - y_min) / img_height

    # Convert the image coordinates to custom coordinates
    custom_x = x * x_scale + x_min
    custom_y = (img_height - y) * y_scale + y_min  # Adjust for coordinate system difference

    # Append the points to the list
    clicked_points.append((x, y))
    custom_coordinates.append((custom_x, custom_y))

    # Print all custom coordinates
    if len(clicked_points) % 1 == 0:
        clear_output(wait=True)
        display(HTML(generate_html()))

        # Print all custom coordinates
        coordinates_str = '\n'.join([f'{coord[0]:.6f}     {coord[1]:.6f}' for coord in custom_coordinates])
        print(coordinates_str)

# Function to handle undo action
def on_undo():
    if clicked_points:
        # Remove the last clicked point and custom coordinates
        clicked_points.pop()
        custom_coordinates.pop()

        # Reprint all remaining custom coordinates
        coordinates_str = '\n'.join([f'{coord[0]:.6f}     {coord[1]:.6f}' for coord in custom_coordinates])
        clear_output(wait=True)
        display(HTML(generate_html()))  # Re-update HTML after undo
        print(coordinates_str)

# Register the click event and undo handlers
output.register_callback('notebook.on_click', on_click)
output.register_callback('notebook.on_undo', on_undo)

# Display the initial image and undo button
display(HTML(generate_html()))